In [1]:
import pandas as pd, numpy as np
import vivarium_output_loader as vol
import lsff_output_processing as lop

!whoami
!date

ndbs
Tue Mar 31 12:36:10 PDT 2020


In [2]:
%load_ext autoreload
%autoreload 2

# Compute change in child mortality rate due to folic acid fortification

That is, compute child_mortality_rate = (deaths among all age groups per year) / (births per year), then see how this changes from baseline to intervention.

## 1. Load data

In [3]:
base_directory = '/share/costeffectiveness/results/vivarium_conic_lsff/v4.0_folic_acid_fortification'

locations_rundates = {
    'Ethiopia': '2020_03_18_23_04_26',
    'India': '2020_03_18_23_04_36',
    'Nigeria': '2020_03_18_23_04_42',
}

In [4]:
data = vol.load_all_transformed_count_data(base_directory, locations_rundates)

In [5]:
[k for k in data.keys() if k[0] == 'nigeria']

[('nigeria', 'gestational_age'),
 ('nigeria', 'transition_count'),
 ('nigeria', 'deaths'),
 ('nigeria', 'state_person_time'),
 ('nigeria', 'births_with_ntd'),
 ('nigeria', 'population'),
 ('nigeria', 'person_time'),
 ('nigeria', 'ylls'),
 ('nigeria', 'ylds'),
 ('nigeria', 'births'),
 ('nigeria', 'birth_weight')]

## 2. Get births and deaths

In [6]:
deaths = data[('nigeria', 'deaths')]
deaths.head()

,year,age_group,sex,cause,fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,diarrheal_diseases,false,death,21,baseline,0.0
1,2020,1_to_4,female,diarrheal_diseases,false,death,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,diarrheal_diseases,false,death,29,baseline,0.0
3,2020,1_to_4,female,diarrheal_diseases,false,death,29,folic_acid_fortification_scale_up,0.0
4,2020,1_to_4,female,diarrheal_diseases,false,death,55,baseline,0.0


In [7]:
births = data[('nigeria', 'births')]
births.head()

,year,sex,fortification_group,measure,input_draw,scenario,value
0,2020,female,false,live_births,21,baseline,20286.0
1,2020,female,false,live_births,21,folic_acid_fortification_scale_up,16490.0
2,2020,female,false,live_births,29,baseline,20252.0
3,2020,female,false,live_births,29,folic_acid_fortification_scale_up,16478.0
4,2020,female,false,live_births,55,baseline,20386.0


## 3. Compute child mortality rate for each year

In [10]:
strata = ['year']
child_mortality_rate = lop.rate_or_ratio(deaths, births, [], strata, 1000)
child_mortality_rate.head()

,year,input_draw,scenario,value
0,2020,21,baseline,81.462547
1,2020,21,folic_acid_fortification_scale_up,81.440320
2,2020,29,baseline,81.064438
3,2020,29,folic_acid_fortification_scale_up,81.064438
4,2020,55,baseline,93.307721


## 4. Compute the change in child mortality with folic acid fortification

In [12]:
delta_child_mortality = lop.averted(child_mortality_rate, 'scenario', 'baseline')
delta_child_mortality.head()

,input_draw,year,value
0,21,2020,0.022227
1,29,2020,0.000000
2,55,2020,0.022079
3,78,2020,0.066025
4,155,2020,0.043648


## 5. Get stats for delta child mortality by year

In [13]:
delta_child_mortality.groupby(['year']).value.describe()

,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
2020,25.0,0.017616,0.017952,0.000000,0.000000,0.021986,0.022133,0.066025
2021,25.0,0.037562,0.020189,0.000000,0.022137,0.033133,0.044304,0.077394
2022,25.0,0.035839,0.025237,0.000000,0.022022,0.033136,0.044146,0.099859
2023,25.0,0.053066,0.027264,0.021960,0.033202,0.044355,0.055487,0.121536
2024,25.0,0.067943,0.034134,0.010983,0.043962,0.077320,0.088553,0.132957
